# Analyze Your Untappd Check-in Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/untappd_details.csv', encoding='utf-8')
df.head()

,date_pacific_tz,beer_name,beer_style,brewery_name,brewery_place,brewery_type,rating,beer_avg_rating,brewery_avg_rating,beer_abv,...,venue_name,venue_type,venue_place,venue_lat,venue_lon,venue_total_checkins,checkin_id,beer_url,brewery_url,venue_url
0,2016-06-09 20:43:09-07:00,Angel City IPA,IPA - American,Angel City Brewery,"Los Angeles, CA United States",Micro Brewery,3.50,3.56,3.47,6.1,...,Boeing Manor,Home (private),"Berkeley, CA",37.8605,-122.271,NaN,321628572,https://untappd.com/b/angel-city-brewery-angel...,https://untappd.com/w/angel-city-brewery/4406,https://untappd.com/v/boeing-manor/509841
1,2016-06-08 01:45:36-07:00,Cascade Premium Light,Lager - Pale,Cascade Brewery Co. (Australia),"South Hobart, Tas. Australia",Macro Brewery,2.00,2.32,2.93,2.6,...,Golden Monkey,"Lounge, Cocktail Bar, Bar","389 Lonsdale St. Melbourne, VIC",-37.8129,144.961,NaN,321044810,https://untappd.com/b/cascade-brewery-co-austr...,https://untappd.com/w/cascade-brewery-co-austr...,https://untappd.com/v/golden-monkey/1201269
2,2016-06-07 22:39:59-07:00,Naked Lager,Lager - Helles,Naked For Satan,Australia,Nano Brewery,3.75,3.47,3.24,4.2,...,Naked For Satan,"Bar, Tapas Restaurant, Lounge","285 Brunswick St. Melbourne, VIC",-37.7988,144.978,NaN,321038174,https://untappd.com/b/naked-for-satan-naked-la...,https://untappd.com/w/naked-for-satan/83569,https://untappd.com/v/naked-for-satan/86227
3,2016-06-07 21:25:57-07:00,Golden Ale,Golden Ale,Venom Brewing,Australia,Micro Brewery,3.25,3.58,3.61,4.8,...,Town Hall Hotel,"Pub, Hotel Bar, Restaurant","166 Johnston St. Fitzroy, VIC",-37.7986,144.981,NaN,321031246,https://untappd.com/b/venom-brewing-golden-ale...,https://untappd.com/w/venom-brewing/219040,https://untappd.com/v/town-hall-hotel/52668
4,2016-06-07 20:37:02-07:00,Fat Yak,Pale Ale - American,Matilda Bay Brewing Company,"Port Melbourne, Vic. Australia",Micro Brewery,4.00,3.39,3.23,4.7,...,Bowl Bowl,Dumpling Restaurant,"88 Smith St Fitzroy, VIC",-37.8056,144.983,NaN,321022902,https://untappd.com/b/matilda-bay-brewing-comp...,https://untappd.com/w/matilda-bay-brewing-comp...,https://untappd.com/v/bowl-bowl/4803853


In [3]:
len(df)

1430

In [4]:
df['rating'].describe()

count    1422.000000
mean        3.590014
std         0.873299
min         0.500000
25%         3.000000
50%         3.500000
75%         4.000000
max         5.000000
Name: rating, dtype: float64

In [5]:
df['beer_brewery'] = df['beer_name'] + ' | ' + df['brewery_name']

In [6]:
unique_beers = df.drop_duplicates(['beer_name', 'brewery_name'])

In [7]:
# the beers i hated the most
df.sort_values('rating', ascending=True)[['beer_name', 'brewery_name', 'rating']].drop_duplicates().head(10)

,beer_name,brewery_name,rating
945,Bud Light Lime Cran-Brrr-Rita,Anheuser-Busch,0.5
403,Coors Light,Coors Brewing Company,0.5
452,Virgin Craft Brewed Lager,MADD Virgin Drinks,0.5
473,Bitburger Drive,Bitburger Braugruppe,0.5
926,Brahma Malzbier,Ambev,0.5
635,Bud Light,Anheuser-Busch,0.5
1225,Natural Light,Anheuser-Busch,0.5
944,Tecate,Cervecería Cuauhtémoc Moctezuma S.A. de C.V.,1.0
1366,Miller Lite,Miller Brewing Company,1.0
1330,Goldbräu / Stiegl Beer,Stieglbrauerei zu Salzburg,1.0


In [8]:
# the beers i loved the most
df.sort_values('rating', ascending=False)[['beer_name', 'brewery_name', 'rating']].drop_duplicates().head(25)

,beer_name,brewery_name,rating
394,Hop Knot,Four Peaks Brewing Company,5.00
1336,Hop Trip,Deschutes Brewery,5.00
486,Pliny the Younger (2015),Russian River Brewing Company,5.00
485,Pliny the Elder,Russian River Brewing Company,5.00
484,Blind Pig IPA,Russian River Brewing Company,5.00
1130,Pabst Blue Ribbon,Pabst Brewing Company,5.00
129,30th Street Pale Ale,Green Flash Brewing Company,5.00
1318,Hofbräu Original,Staatliches Hofbräuhaus München,5.00
134,Pale Ale,Sierra Nevada Brewing Co.,5.00
1325,Proving Ground IPA,Magnolia Brewing Company,5.00


In [9]:
# the beers i've had that the public has loved the most
df.sort_values('beer_avg_rating', ascending=False)[['beer_name', 'brewery_name', 'beer_avg_rating', 'rating']].drop_duplicates().head(10)

,beer_name,brewery_name,beer_avg_rating,rating
486,Pliny the Younger (2015),Russian River Brewing Company,4.71,5.00
487,Pliny the Younger,Russian River Brewing Company,4.70,5.00
914,Pliny the Elder,Russian River Brewing Company,4.57,5.00
885,Vintage (Barrel No 144) (2010),Brouwerij Rodenbach,4.37,4.00
51,Sucaba,Firestone Walker Brewing Company,4.37,3.75
1254,Stone Enjoy By 04.01.13 IPA,Stone Brewing,4.35,4.50
311,Hop Venom,Boneyard Beer Company,4.20,3.75
1074,Blind Pig IPA,Russian River Brewing Company,4.20,5.00
640,Xocoveza Mocha Stout,Stone Brewing,4.16,2.00
196,Born Yesterday Pale Ale (2015),Lagunitas Brewing Company,4.14,4.25


In [10]:
# the beers i've had that the public has hated the most
df.sort_values('beer_avg_rating', ascending=True)[['beer_name', 'brewery_name', 'beer_avg_rating', 'rating']].drop_duplicates().head(10)

,beer_name,brewery_name,beer_avg_rating,rating
452,Virgin Craft Brewed Lager,MADD Virgin Drinks,1.14,0.50
468,St. Pauli Girl NA,Brauerei Beck,1.96,2.00
1225,Natural Light,Anheuser-Busch,1.97,0.50
691,Big Flats 1901,"Winery Exchange, World Brews",2.08,1.50
926,Brahma Malzbier,Ambev,2.13,0.50
431,Dali V8,Dali Beer Group,2.14,2.25
925,Skol,Ambev,2.18,2.00
473,Bitburger Drive,Bitburger Braugruppe,2.19,0.50
430,Snow,China Resources Snow Breweries,2.19,2.50
432,Dali V6,Dali Beer Group,2.22,1.75


In [11]:
# beers i've had that the public has had the most
df.sort_values('beer_total_checkins', ascending=False)[['beer_name', 'brewery_name', 'beer_total_checkins', 'rating']].drop_duplicates().head(10)

,beer_name,brewery_name,beer_total_checkins,rating
1002,Guinness Draught,Guinness,1180000,4.0
635,Bud Light,Anheuser-Busch,1090000,0.5
1366,Miller Lite,Miller Brewing Company,997301,1.0
594,Traditional Lager,Yuengling Brewery,941923,3.5
403,Coors Light,Coors Brewing Company,874086,0.5
353,IPA,Lagunitas Brewing Company,781131,4.0
943,Two Hearted Ale,Bell's Brewery,764932,3.0
1160,Pabst Blue Ribbon,Pabst Brewing Company,703250,5.0
276,Heineken,Heineken,702854,3.0
1028,Belgian White,Blue Moon Brewing Company,673470,3.0


In [12]:
# places i've checked-in that the public has checked-in the most
df.sort_values('venue_total_checkins', ascending=False)[['beer_name', 'venue_name', 'venue_total_checkins']].drop_duplicates().head(10)

,beer_name,venue_name,venue_total_checkins
0,Angel City IPA,Boeing Manor,NaN
1,Cascade Premium Light,Golden Monkey,NaN
2,Naked Lager,Naked For Satan,NaN
3,Golden Ale,Town Hall Hotel,NaN
4,Fat Yak,Bowl Bowl,NaN
5,Pale Ale,Nieuw Amsterdam,NaN
6,Pale Ale,Great Ocean Road Resort,NaN
7,Super Crisp Lager,NaN,NaN
8,American Pale Ale,Trinity Bar,NaN
9,IPA,Trinity Bar,NaN


In [13]:
# what styles have i had the most?
df['beer_style'].value_counts().head(10)

IPA - American                       264
Pale Ale - American                  142
Lager - North American Adjunct       124
Lager - Pale                          79
Pilsner - German                      62
Pilsner - Other                       59
Pilsner - Czech                       42
Red Ale - American Amber / Red        36
Kölsch                                33
IPA - Session / India Session Ale     32
Name: beer_style, dtype: int64

In [14]:
# what basic styles have i had the most?
df['beer_style'].map(lambda x: unicode(x).split(' - ')[0]).value_counts().head(10)

IPA                              359
Lager                            314
Pale Ale                         172
Pilsner                          163
Red Ale                           45
Kölsch                            33
California Common                 22
Golden Ale                        20
Extra Special / Strong Bitter     18
Stout                             18
Name: beer_style, dtype: int64

In [15]:
# what beers have i checked-in the most?
df['beer_brewery'].value_counts().head(10)

Pabst Blue Ribbon | Pabst Brewing Company             44
Pale Ale | Sierra Nevada Brewing Co.                  21
Trumer Pils | Trumer                                  17
Hop Knot | Four Peaks Brewing Company                 17
Anchor Steam Beer | Anchor Brewing Company            16
IPA | Lagunitas Brewing Company                       15
Hamm's Premium | Miller Brewing Company               11
Kölsch | Jupiter                                      10
Hofbräu Original | Staatliches Hofbräuhaus München    10
Summer Solstice | Anderson Valley Brewing Company     10
Name: beer_brewery, dtype: int64

In [16]:
# what breweries have i checked into the most?
df['brewery_name'].value_counts().head(10)

Sierra Nevada Brewing Co.          71
Four Peaks Brewing Company         56
Pabst Brewing Company              55
Lagunitas Brewing Company          37
Deschutes Brewery                  34
Jupiter                            31
Ninkasi Brewing Company            22
SanTan Brewing Company             19
Anchor Brewing Company             19
Anderson Valley Brewing Company    18
Name: brewery_name, dtype: int64

In [17]:
# what breweries have i had the greatest number of unique beers from?
unique_beers['brewery_name'].value_counts().head(10)

Sierra Nevada Brewing Co.      38
Lagunitas Brewing Company      18
Deschutes Brewery              16
Four Peaks Brewing Company     15
Stone Brewing                  13
Ninkasi Brewing Company        13
SanTan Brewing Company         12
New Belgium Brewing Company    11
Jupiter                         8
McMenamins                      8
Name: brewery_name, dtype: int64

In [18]:
# what unique beers have i had from four peaks?
unique_beers[unique_beers['brewery_name']=='Four Peaks Brewing Company'][['date_pacific_tz', 'beer_name', 'rating']]

,date_pacific_tz,beer_name,rating
41,2016-05-02 20:17:30-07:00,Gumleaf Schooner,4.50
42,2016-05-02 19:23:14-07:00,Short Hop,4.50
45,2016-05-01 19:05:37-07:00,Hop Knot,5.00
47,2016-05-01 16:39:45-07:00,8th Street Pale Ale,4.00
184,2015-12-26 22:03:33-08:00,Raj IPA,4.00
187,2015-12-26 18:24:05-08:00,White Ale,2.75
199,2015-12-22 22:01:59-08:00,Double Knot,4.00
265,2015-10-03 14:51:03-07:00,Lefty's Lager,3.75
266,2015-10-03 14:49:41-07:00,Western Rivers English Summer Ale,3.75
267,2015-10-03 14:01:10-07:00,Single Tank Series: Hoppy Pils,4.50
